In [1]:
from problem_model import forest_problem_vaaler, simple_forest_problem

from methods import binary_nsga3

In [2]:
problem = simple_forest_problem()

solver, publisher = binary_nsga3(problem=problem)

In [3]:
result = solver()

ValueError: operands could not be broadcast together with shapes (500,5) (60,) 